In [2]:
import pandas as pd
from tqdm import tqdm
nodes = pd.read_csv("nodes_20250223_093508.csv")
links = pd.read_csv("links_20250223_093508.csv")
display(nodes.groupby("type").agg("count"))
print(links.shape[0], nodes.shape[0])
pd.set_option('display.max_rows', None)
import networkx as nx
from pyvis.network import Network

# Add source and target type to links
def node_type(x):
    return nodes[nodes.id==x].iloc[0].type
links= links[links.type!="testrel"]
links["source_type"] = links.source.apply(node_type)
links["target_type"] = links.target.apply(node_type)
links.head(2)


,id,label
type,,
Event,1,1
Expression,3111,3111
Institution,80,80
Manuscript,2377,2377
ManuscriptPart,2664,2664
Person,4291,4291
Place,545,545
Work,2198,2198


25458 15267


,id,source,target,type,source_type,target_type
0,1,1,2,son of,Person,Person
1,2,1,2,studied with,Person,Person


 I need to ask you to send me some visualisations for a presentation I have in a few weeks. Mainly I need connections to "Place" and "Expression" and also "people" connecte to  "Expression" as "copied by". I also need the connection of all expression which have connection to place, including the relation between expression and manuscript, expression and work and manuscript to institution. Please, do this with the existing data in the old database.


## Keep only links that have a pathway from Expression to Person or Expression to Place

### 1. Expression copied by Person
    - Remove all links involving person not in above link

In [3]:
person_expression_copied_by = links[links.type=="copied by"].target
unwanted_persons = nodes[(nodes.type=="Person") &(~nodes.id.isin(person_expression_copied_by))]
unwanted_persons.shape[0]
unwanted_person_links = links[links.source.isin(unwanted_persons.id) | (links.target.isin(unwanted_persons.id))]
print(unwanted_person_links.shape[0])
selected_links = links[~links.id.isin(unwanted_person_links.id)]
print(selected_links.shape[0])

8356
17101


### 2. Institution located in Place

In [4]:
institutions_located_in_place = selected_links[(selected_links.source_type=="Institution")&((selected_links.type=="located at")|(selected_links.type=="located in"))]
print(institutions_located_in_place.shape[0])
unwanted_institutions = nodes[(nodes.type=="Institution") & (~nodes.id.isin(institutions_located_in_place.source))]
print(unwanted_institutions.shape[0])
unwanted_institution_links = links[(links.source.isin(unwanted_institutions.id)) | (links.target.isin(unwanted_institutions.id))]
print(unwanted_institution_links.shape[0])
selected_links = selected_links[~selected_links.id.isin(unwanted_institution_links.id)]
print(selected_links.shape[0])

57
24
49
17063


### 3. Manuscript Part acquired in Place

In [5]:
# manuscript_place_place_of_acquisiton = selected_links[(selected_links.target_type=="ManuscriptPart")&((selected_links.type=="place of acquisition"))]
# print(manuscript_place_place_of_acquisiton.shape[0])
# unwanted_manuscript_parts = nodes[(nodes.type=="ManuscriptPart")&(~nodes.id.isin(manuscript_place_place_of_acquisiton.target))]
# print(unwanted_manuscript_parts.shape[0])
# unwanted_manuscript_part_links = links[(links.source.isin(unwanted_manuscript_parts.id)) | (links.target.isin(unwanted_manuscript_parts.id))]
# print(unwanted_manuscript_part_links.shape[0])
# selected_links = selected_links[~selected_links.id.isin(unwanted_manuscript_part_links.id)]
# print(selected_links.shape[0])

### 4. Manuscripts containing Expressions

In [6]:
manuscript_contains_expression = selected_links[(selected_links.target_type=="Expression")&(selected_links.type=="contains")]
unwanted_manuscripts = nodes[(nodes.type=="Manuscript")&(~nodes.id.isin(manuscript_contains_expression.source))]
print(manuscript_contains_expression.shape[0], unwanted_manuscripts.shape[0])
unwanted_manuscript_links = links[(links.source.isin(unwanted_manuscripts.id)) | (links.target.isin(unwanted_manuscripts.id))]
print(unwanted_manuscript_links.shape[0])
selected_links = selected_links[~selected_links.id.isin(unwanted_manuscript_links.id)]
print(selected_links.shape[0])


3101 88
215
16854


### 4.1 Manuscript-Institutions


In [7]:

institutions_manuscripts =  selected_links[(selected_links.target_type=="Institution")&(selected_links.source_type=="Manuscript")]
unwanted_institutions = nodes[(nodes.type=="Institution")&(~nodes.id.isin(institutions_manuscripts.source))]
unwanted_institution_links = links[(links.source.isin(unwanted_institutions.id)) | (links.target.isin(unwanted_institutions.id))]
print(unwanted_institution_links.shape[0])
selected_links = selected_links[~selected_links.id.isin(unwanted_institution_links.id)]
print(selected_links.shape[0])


2421
14540


## 5. Works Place Relations

In [8]:
work_place_relations = selected_links[(selected_links.target_type=="Work")&(selected_links.source_type=="Place")]
print(work_place_relations.shape[0])
unwanted_works = nodes[(nodes.type=="Work") & (~nodes.id.isin(work_place_relations.target))]
print(unwanted_works.shape[0])
unwanted_work_links = links[(links.source.isin(unwanted_works.id)) | (links.target.isin(unwanted_works.id))]
print(unwanted_work_links.shape[0])
selected_links = selected_links[~selected_links.id.isin(unwanted_work_links.id)]
print(selected_links.shape[0])

48
2154
8364
8313


## 6. Expression-Manuscript-Place and Expression-Manuscript-Institution-Place

Keep only those Expression-Manuscripts where Manuscripts have a relation to place


In [9]:
manuscript_place_links = selected_links[(selected_links.source_type=="Manuscript")&((selected_links.target_type=="Place")|(selected_links.source_type=="Institution"))]
print(manuscript_place_links.shape[0])

unwanted_manuscripts = nodes[(nodes.type=="Manuscript")&(~nodes.id.isin(manuscript_place_links.source))]
unwanted_manuscript_links = links[(links.source.isin(unwanted_manuscripts.id)) | (links.target.isin(unwanted_manuscripts.id))]
print(unwanted_manuscript_links.shape[0])
selected_links = selected_links[~selected_links.id.isin(unwanted_manuscript_links.id)]
print(selected_links.shape[0])


343
11196
3371


### 7. Remove links between the same types

In [10]:
selected_links = selected_links[selected_links.source_type!=selected_links.target_type]
print(selected_links.shape[0])

2974


### 8. Remove Persons

In [11]:
# selected_links = selected_links[selected_links.source_type!="Person"]
# selected_links = selected_links[selected_links.target_type!="Person"]
# print(selected_links.shape[0])

In [12]:
display(selected_links.groupby(["source_type", "target_type"]).agg("count"))
print(selected_links.shape[0])

id  source  target  type
source_type target_type                               
Expression  ManuscriptPart     2       2       2     2
            Person          1232    1232    1232  1232
            Work             126     126     126   126
Manuscript  Expression       438     438     438   438
            ManuscriptPart   410     410     410   410
            Person             2       2       2     2
            Place            343     343     343   343
            Work              16      16      16    16
Person      ManuscriptPart    31      31      31    31
            Place             74      74      74    74
            Work               7       7       7     7
Place       Expression       237     237     237   237
            ManuscriptPart     8       8       8     8
            Work              48      48      48    48

2974


In [13]:
display(selected_links.groupby(["type"]).agg({"id":"count"}).sort_values("id", ascending=False).head(10))



,id
type,
copied by,1217
contains,848
copied in,322
place of copy of,237
a copy of,126
place of composition of,47
bought in,21
lived in,18
mentioned in,18


In [14]:
def get_link(node):
    return f"<a href='https://nomansland-dev.acdh-ch-dev.oeaw.ac.at/apis/apis_ontology.{node.type.lower()}/{node.id}>{node.label}</a>"


def make_graph(nodes_df, links_df, html_out="pyvis.html"):
    ICONS = {'person': '\uf183',
 'place': '\uf3c5',
 'institution': '\uf19c',
 'event': '\uf0f3',
 'work': '\uf02d',
 'expression': '\uf219',
 'manuscript': '\uf70e',
 'manuscriptpart': '\ue698'}

    COLOURS = {
    'person': 'orange',     
    'place': 'green',       
    'institution': 'purple',
    'work': 'red',          
    'expression': 'blue',
    'manuscript': 'brown',
    'manuscriptpart': 'brown'
    }
    def get_node(id):
        return nodes[nodes_df.id==id].iloc[0]

    G = nx.Graph()
        
    for i, l in tqdm(links_df.iterrows(), total=links_df.shape[0]):
        src = get_node(l.source)
        tgt = get_node(l.target)
        src_url = f""
        tgt_url = f"https://nomansland-dev.acdh-ch-dev.oeaw.ac.at/apis/apis_ontology.{tgt.type.lower()}/{tgt.id}"
    
        source_target_type = " - ".join(sorted([src.type, tgt.type]))
        relation_pair = 
        G.add_edge(l.source, l.target, group=l.type, source_target_type=source_target_type,  title=f"{src.label} {l.type} {tgt.label}")
        G.add_node(src.id, title=src.label, label=src.label, group=src.type.lower(),shape="icon")
        G.add_node(tgt.id, title=tgt.label, label=tgt.label, group=tgt.type.lower(),shape="icon")
    
    net = Network(notebook=True,height="90vh", width="100%", filter_menu=True,layout=True, neighborhood_highlight=True)
    net.from_nx(G)
    # # Add nodes with size based on degree (number of connections)
    # for node in G.nodes():
    #     degree = G.degree(node)  # Get number of edges for this node
    #     node_data = get_node(node)
    #     net.add_node(node,#shape="icon", size=degree * 5, 
    #                  title=node_data.label, group=node_data.type)  # Scale size (adjust multiplier as needed)
    
    net.set_options("""
     var options={"nodes": {
        "borderWidth": "null",
        "borderWidthSelected": "null",
        "opacity": "null",
        "shapeProperties": {
          "interpolation": "false"
        },
        "size": 50
      },
      "groups": {},
      "edges": {
          "width":10,
          "smooth":"false",
          "color": "blue"
      },
      "interaction": {
        "navigationButtons": "true"
      },
      "physics": {
        "stabilization": { "iterations": 100, "fit": "true" },
        "barnesHut": { "gravitationalConstant": -8000 },   
        "minVelocity": 0.75
      },
        "configure": {"enabled": "true", "showButton": "true", 
                      "filter": ["physics", "nodes", "edges"]}
    
    }
    """)
    
    for g in nodes.type.unique():
        net.options["groups"][g.lower()] =  {
                    "shape": 'icon',
                    "size": 50,
                    "icon": {
                        "face": 'FontAwesome',
                        "code": ICONS.get(g.lower()),
                        "color": COLOURS.get(g.lower())
                    }
        }
    
    net.write_html(html_out)
    html_str = net.html.replace(
      '<head>',
       '<head><link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.7.2/css/all.min.css">').replace('"false"',
                                                                                                                                 "false").replace('"true"', 
                                                                                                                                 "true").replace('"null"', 
                                                                                                                                                 "null")
    
    # net.save_graph(output_file)
    with open(html_out, "w") as f:
        f.write(html_str)

    print("The visualisation is saved in ", html_out)

In [17]:
links_sans_persons = selected_links[(selected_links.target_type!="Person")&(selected_links.source_type!="Person")]
links_sans_persons.shape[0]

1628

In [19]:
make_graph(nodes, selected_links, "../viz/expressions-persons-places.html")
make_graph(nodes, links_sans_persons, "../viz/expressions-places.html")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2974/2974 [00:02<00:00, 1112.64it/s]


The visualisation is saved in  ../viz/expressions-persons-places.html


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1628/1628 [00:01<00:00, 1154.87it/s]


The visualisation is saved in  ../viz/expressions-places.html


In [20]:
selected_links.to_csv("expressions-persons-places.csv", index=False)

In [21]:
links_sans_persons.to_csv("expressions-places.csv", index=False)